In [1]:
import pickle
import sys
import math
from random import random

with open('../data/test.pickle', 'rb') as fd:
    test_cases = pickle.load(fd)

with open('../data/map.pickle', 'rb') as fm:
    mp = pickle.load(fm)

with open('../data/dict.pickle', 'rb') as fd:
    dc = pickle.load(fd)

print('Initialized')

Initialized


In [2]:
def strictCmp(a, b, p):
    return 1. if a == b else 0

def rougeCmp(a, b, p):
    if len(a) != len(b) or len(a) < 1:
        return 0.
    a = '_' + a
    b = '_' + b
    n = len(a)
    f = []
    for i in range(n):
        f.append([ 0 ] * n)
    for i in range(1, n):
        for j in range(1, n):
            f[i][j] = max(f[i][j], f[i - 1][j], f[i][j - 1])
            if a[i] == b[j]:
                f[i][j] = max(f[i][j], f[i - 1][j - 1] + 1)
    n -= 1
    lcs = f[n][n]
    return 1. * lcs / max(n, 1.)

def genResStr(tot, rat1, rat2):
    return 'Precision %.6f ROUGE-L %.6f' % ( 1. * rat1 / tot, 1. * rat2 / tot )

def test():
    tot = 0
    res_strict = 0
    res_lcs = 0
    for i in range(len(test_cases)):
        c = test_cases[i]
        ans = pinyin2text(c['pinyin'])
        tot += 1
        res_strict += strictCmp(ans, c['text'], c['pinyin'])
        res_lcs += rougeCmp(ans, c['text'], c['pinyin'])
        if i % 64 == 0:
            print('[Case 0 - %d] %s' % (i, genResStr(tot, res_strict, res_lcs)))

    print('[Fin] %s' % genResStr(tot, res_strict, res_lcs))

In [13]:
max_iter = 100
stable_iter = 3
eps = .000001
innocent = math.exp(eps)

def evaluateAns(py, ans):
    n = len(py)
    def getLnkVal(a, b, pin, direct):
        global dc
        global mp
        totl = 0
        for i in dc[pin]:
            if a in mp[direct] and i in mp[direct][a]:
                totl += mp[direct][a][i]
        if a in mp[direct] and b in mp[direct][a]:
            curl = mp[direct][a][b]
        else:
            curl = 0
        lnk_v = curl / (totl + eps) + eps
        return lnk_v
    s = 1.
    for i in range(0, n - 1):
        if i + 1 < n - 1:
            s *= getLnkVal(ans[i], ans[i + 1], py[i + 1], 'fw')
        if i > 0:
            s *= getLnkVal(ans[i + 1], ans[i], py[i], 'bw')
    return s

def randomFetchChar(prv_chr, py, direct, is_head, last_ans, p_cnt):
    wl = {}
    if not py in dc:
        return '<unk>'
    sum_p = 0
    max_p = 0
    totl = 0
    for i in dc[py]:
        if prv_chr in mp[direct] and i in mp[direct][prv_chr]:
            totl += mp[direct][prv_chr][i]
    mul_x = 10. / (totl + .001)
    for i in dc[py]:
        if prv_chr in mp[direct] and i in mp[direct][prv_chr]:
            # print('%s = %.5f / %d * 10 (%.5lf)' % (i, mp[direct][prv_chr][i], totl, mul_x))
            wl[i] = math.exp(mp[direct][prv_chr][i] * mul_x)
        elif i != py:
            wl[i] = innocent
        else:
            continue
        max_p = max(wl[i], max_p)
        sum_p += wl[i]
    if last_ans != '<todo>':
        if not last_ans in wl:
            wl[last_ans] = innocent
        if is_head:
            p_prv = max_p * math.exp(p_cnt ** 2)
        else:
            p_prv = wl[last_ans] * math.exp(p_cnt)
        wl[last_ans] += p_prv
        sum_p += p_prv
    chosen_p = random() * sum_p
    for i in wl:
        chosen_p -= wl[i]
        if chosen_p <= 0.:
            return i
    return '<err>'

def cmbAns(ans, plc = ''):
    return plc.join(ans[1 : -1])

def printAnsWithVal(py, ans):
    print('%s %e' % (cmbAns(ans), evaluateAns(py, ans)))

def pinyin2text(raw_line):
    a = [ '' ] + raw_line.strip().rstrip().split(' ') # For same id
    n = len(a) - 1
    ans = [ 'start' ] + [ '<todo>' ] * n + [ 'end' ]
    for cnt_iter in range(0, max_iter):
        valid_iter = False
        for i in range(1, n + 1):
            g_chr = randomFetchChar(ans[i - 1], a[i], 'fw', i == 1, ans[i], ctn_stable)
            if g_chr != ans[i]:
                valid_iter = True
            ans[i] = g_chr
        printAnsWithVal(a, ans)

        #print('%s at %d' % (' '.join(ans[1 : n + 1]), cnt_iter))
    return ''.join(ans[1 : n + 1])



In [14]:
pinyin2text('wo mei you shuo hua')

我没有数花 1.436503e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有槊骅 6.984161e-19
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数砉 4.146257e-07
我没有数化 3.257777e-01
我没有数华 1.575582e-01
我没有数化 3.257777e-01
我没有数画 1.091124e-01
我没牖说画 4.990962e-25
我没有数话 2.153951e-04
我没有说华 5.271683e-02
我没有数华 1.575582e-01
我没有说华 5.271683e-02
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数花 1.436503e-01
呙鹛由朔桦 8.484272e-31
我没有数花 1.436503e-01
我没有说花 5.375797e-02
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数骅 4.146257e-07
我没有数骅 4.146257e-07
我没有数砉 4.146257e-07
我没有数化 3.257777e-01
我没有数划 2.430568e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01
我没有数化 3.257777e-01


'我没有数化'

In [4]:
test()

[Case 0 - 0] Precision 0.000000 ROUGE-L 0.578947
[Case 0 - 64] Precision 0.076923 ROUGE-L 0.482520
[Case 0 - 128] Precision 0.062016 ROUGE-L 0.512386
[Case 0 - 192] Precision 0.046632 ROUGE-L 0.494569
[Case 0 - 256] Precision 0.050584 ROUGE-L 0.461987
[Case 0 - 320] Precision 0.049844 ROUGE-L 0.469955
[Case 0 - 384] Precision 0.051948 ROUGE-L 0.475007
[Case 0 - 448] Precision 0.060134 ROUGE-L 0.481001
[Case 0 - 512] Precision 0.058480 ROUGE-L 0.474806
[Case 0 - 576] Precision 0.058925 ROUGE-L 0.473528
[Case 0 - 640] Precision 0.062402 ROUGE-L 0.475625
[Case 0 - 704] Precision 0.062411 ROUGE-L 0.476508
[Case 0 - 768] Precision 0.059818 ROUGE-L 0.478143
[Fin] Precision 0.058971 ROUGE-L 0.477700
